In [12]:
%load_ext autoreload
%autoreload 2

from src.data_processing.read import read_device_status_from_zip, parse_date_columns
from src.configurations import Configuration


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:

zip_id = '00221634'
config = Configuration()
file = config.data_dir + '\\' +zip_id + '.zip'
read_record = read_device_status_from_zip(file,config)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
00221634


In [23]:

time_cols = [c for c in config.time_cols() if c in read_record.df.columns]

df = read_record.df[time_cols].melt()
df['datetime'] = parse_date_columns(df['value'])
df['timezone'] = df['datetime'].dt.tz
df['timezone'].unique()


array([datetime.timezone.utc], dtype=object)